# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [3]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ast

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.cluster import KMeans

from sklearn.metrics import roc_auc_score, fbeta_score, accuracy_score, precision_score, recall_score


# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [3]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Reading in Datasets (AZDIAS & CUSTOMERS)

**AZDIAS** Dataset

In [4]:
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [5]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(93), object(6)
memory usage: 2.4+ GB


In [6]:
azdias.shape

(891221, 366)

In [7]:
azdias.describe()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
count,8.912210e+05,891221.000000,817722.000000,817722.000000,81058.000000,29499.000000,6170.000000,1205.000000,628274.000000,798073.000000,...,770025.000000,815304.000000,815304.000000,815304.000000,783619.000000,817722.000000,798073.000000,891221.000000,891221.000000,891221.000000
mean,6.372630e+05,-0.358435,4.421928,10.864126,11.745392,13.402658,14.476013,15.089627,13.700717,8.287263,...,2.417322,6.001214,7.532130,5.945972,3.933406,7.908791,4.052836,3.362438,1.522098,2.777398
std,2.572735e+05,1.198724,3.638805,7.639683,4.097660,3.243300,2.712427,2.452932,5.079849,15.628087,...,1.166572,2.856091,3.247789,2.771464,1.964701,1.923137,1.949539,1.352704,0.499512,1.068775
min,1.916530e+05,-1.000000,1.000000,0.000000,2.000000,2.000000,4.000000,7.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
25%,4.144580e+05,-1.000000,1.000000,0.000000,8.000000,11.000000,13.000000,14.000000,11.000000,1.000000,...,2.000000,3.000000,5.000000,4.000000,2.000000,8.000000,3.000000,3.000000,1.000000,2.000000
50%,6.372630e+05,-1.000000,3.000000,13.000000,12.000000,14.000000,15.000000,15.000000,14.000000,4.000000,...,2.000000,6.000000,8.000000,6.000000,4.000000,9.000000,3.000000,3.000000,2.000000,3.000000
75%,8.600680e+05,-1.000000,9.000000,17.000000,15.000000,16.000000,17.000000,17.000000,17.000000,9.000000,...,3.000000,9.000000,10.000000,8.000000,6.000000,9.000000,5.000000,4.000000,2.000000,4.000000
max,1.082873e+06,3.000000,9.000000,21.000000,18.000000,18.000000,18.000000,18.000000,25.000000,595.000000,...,4.000000,11.000000,13.000000,11.000000,6.000000,9.000000,8.000000,6.000000,2.000000,9.000000


**CUSTOMERS** Dataset

In [8]:
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [9]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(8)
memory usage: 539.5+ MB


In [10]:
customers.shape

(191652, 369)

In [11]:
customers.describe()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
count,191652.000000,191652.000000,145056.000000,145056.000000,11766.000000,5100.000000,1275.000000,236.000000,139810.000000,141725.000000,...,143781.000000,143781.000000,143781.000000,137910.000000,145056.000000,141725.000000,191652.000000,191652.000000,191652.000000,191652.000000
mean,95826.500000,0.344359,1.747525,11.352009,12.337243,13.672353,14.647059,15.377119,10.331579,4.965863,...,4.374417,4.564769,3.168868,4.152716,8.646371,3.723133,2.576806,0.090247,1.376432,3.060907
std,55325.311233,1.391672,1.966334,6.275026,4.006050,3.243335,2.753787,2.307653,4.134828,14.309694,...,2.924355,2.887035,2.233516,1.974375,1.154001,2.095540,1.168486,0.286536,0.484492,1.086254
min,1.000000,-1.000000,1.000000,0.000000,2.000000,2.000000,5.000000,8.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
25%,47913.750000,-1.000000,1.000000,8.000000,9.000000,11.000000,13.000000,14.000000,9.000000,1.000000,...,2.000000,2.000000,1.000000,2.000000,9.000000,2.000000,1.000000,0.000000,1.000000,3.000000
50%,95826.500000,0.000000,1.000000,11.000000,13.000000,14.000000,15.000000,16.000000,10.000000,1.000000,...,4.000000,4.000000,3.000000,5.000000,9.000000,3.000000,3.000000,0.000000,1.000000,3.000000
75%,143739.250000,2.000000,1.000000,16.000000,16.000000,16.000000,17.000000,17.000000,13.000000,4.000000,...,7.000000,7.000000,4.000000,6.000000,9.000000,5.000000,3.000000,0.000000,2.000000,4.000000
max,191652.000000,3.000000,9.000000,21.000000,18.000000,18.000000,18.000000,18.000000,25.000000,523.000000,...,11.000000,13.000000,11.000000,6.000000,9.000000,8.000000,6.000000,1.000000,2.000000,9.000000


### A. Data Preprocessing

Preprocessing the AZDIAS Dataset

### 1. Drop features (columns) in AZDIAS not described in the Attributes Excel Sheet (attributes_info)

I have manually created a csv file ('attributes.csv') which can be used to easily identify and manipulate the attributes describred in the DIAS Attributes - Values 2017.xlsx file. This csv file, attributes.csv, which you can visualise the first 5 rows of below, is composed of 3 columns: Attribute (name of the attribute, i.e. some of the columns in AZDIAS dataset), Type ('categorical', 'ordinal', 'numeric') and Unknown_Values (values for each feature/column of the dataset that have the meaning of 'unknown'; see the 'Meaning' Column of the DIAS Attributes - Values 2017.xlsx file). 

This manually created csv file contains 335 rows (including the first row 'Attribute;Type;Value'), so there are a total of 334 features described. Thus, AZDIAS dataset (366 columns) contains 32 additional features not found in the attributes.csv description file.

In [12]:
attributes_info = pd.read_csv('./attributes.csv', sep=';') 

In [13]:
attributes_info.head()

,Attribute,Type,Unknown_Values
0,AGER_TYP,categorical,"[-1,0]"
1,ALTERSKATEGORIE_FEIN,ordinal,"[-1,0.9]"
2,ALTERSKATEGORIE_GROB,ordinal,"[-1,0,9]"
3,ALTER_HH,ordinal,[0]
4,ANREDE_KZ,categorical,"[-1,0]"


In [14]:
features_in_attributes_info = attributes_info['Attribute'].tolist()

In [15]:
features_in_attributes_info

['AGER_TYP',
 'ALTERSKATEGORIE_FEIN',
 'ALTERSKATEGORIE_GROB',
 'ALTER_HH',
 'ANREDE_KZ',
 'ANZ_HAUSHALTE_AKTIV',
 'ANZ_HH_TITEL',
 'ANZ_KINDER',
 'ANZ_PERSONEN',
 'ANZ_STATISTISCHE_HAUSHALTE',
 'ANZ_TITEL',
 'ARBEIT',
 'BALLRAUM',
 'CAMEO_DEUG_2015',
 'CAMEO_DEU_2015',
 'CAMEO_INTL_2015',
 'CJT_GESAMTTYP',
 'D19_BANKEN_ANZ_12',
 'D19_BANKEN_ANZ_24',
 'D19_BANKEN_DATUM',
 'D19_BANKEN_DIREKT',
 'D19_BANKEN_GROSS',
 'D19_BANKEN_LOKAL',
 'D19_BANKEN_OFFLINE_DATUM',
 'D19_BANKEN_ONLINE_DATUM',
 'D19_BANKEN_ONLINE_QUOTE_12',
 'D19_BANKEN_REST',
 'D19_BEKLEIDUNG_GEH',
 'D19_BEKLEIDUNG_REST',
 'D19_BILDUNG',
 'D19_BIO_OEKO',
 'D19_BUCH_CD',
 'D19_DIGIT_SERV',
 'D19_DROGERIEARTIKEL',
 'D19_ENERGIE',
 'D19_FREIZEIT',
 'D19_GARTEN',
 'D19_GESAMT_ANZ_12',
 'D19_GESAMT_ANZ_24',
 'D19_GESAMT_DATUM',
 'D19_GESAMT_OFFLINE_DATUM',
 'D19_GESAMT_ONLINE_DATUM',
 'D19_GESAMT_ONLINE_QUOTE_12',
 'D19_HANDWERK',
 'D19_HAUS_DEKO',
 'D19_KINDERARTIKEL',
 'D19_KONSUMTYP',
 'D19_KOSMETIK',
 'D19_LEBENSMITTEL',
 

In [16]:
azdias_columns_to_drop = list(set(azdias) - set(features_in_attributes_info))

In [17]:
# columns in AZDIAS dataset but not described in attributes_info: to drop
azdias_columns_to_drop

['LNR',
 'FIRMENDICHTE',
 'CJT_TYP_1',
 'UMFELD_ALT',
 'VK_DHT4A',
 'RT_UEBERGROESSE',
 'ALTER_KIND2',
 'VK_ZG11',
 'CJT_TYP_5',
 'D19_KONSUMTYP_MAX',
 'EXTSEL992',
 'VERDICHTUNGSRAUM',
 'CJT_TYP_3',
 'RT_KEIN_ANREIZ',
 'VHN',
 'ALTER_KIND3',
 'ALTER_KIND1',
 'RT_SCHNAEPPCHEN',
 'EINGEFUEGT_AM',
 'CJT_KATALOGNUTZER',
 'D19_LETZTER_KAUF_BRANCHE',
 'VHA',
 'UMFELD_JUNG',
 'CJT_TYP_4',
 'AKT_DAT_KL',
 'VK_DISTANZ',
 'CJT_TYP_6',
 'KOMBIALTER',
 'ALTER_KIND4',
 'GEMEINDETYP',
 'STRUKTURTYP',
 'CJT_TYP_2']

In [19]:
azdias.drop(labels=azdias_columns_to_drop, axis=1, inplace=True)

In [20]:
# check the remaining columns
azdias.columns

Index(['AGER_TYP', 'ALTER_HH', 'ALTERSKATEGORIE_FEIN', 'ANZ_HAUSHALTE_AKTIV',
       'ANZ_HH_TITEL', 'ANZ_KINDER', 'ANZ_PERSONEN',
       'ANZ_STATISTISCHE_HAUSHALTE', 'ANZ_TITEL', 'ARBEIT',
       ...
       'SOHO_KZ', 'TITEL_KZ', 'UNGLEICHENN_FLAG', 'VERS_TYP', 'W_KEIT_KIND_HH',
       'WOHNDAUER_2008', 'WOHNLAGE', 'ZABEOTYP', 'ANREDE_KZ',
       'ALTERSKATEGORIE_GROB'],
      dtype='object', length=334)

In [21]:
# visualing the first few lines of AZDIAS dataset again
azdias.head()

,AGER_TYP,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,...,SOHO_KZ,TITEL_KZ,UNGLEICHENN_FLAG,VERS_TYP,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-1,NaN,NaN,NaN,3,1,2
1,-1,0.0,21.0,11.0,0.0,0.0,2.0,12.0,0.0,3.0,...,1.0,0.0,1.0,2,3.0,9.0,4.0,5,2,1
2,-1,17.0,17.0,10.0,0.0,0.0,1.0,7.0,0.0,3.0,...,0.0,0.0,0.0,1,3.0,9.0,2.0,5,2,3
3,2,13.0,13.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,1,NaN,9.0,7.0,3,2,4
4,-1,20.0,14.0,3.0,0.0,0.0,4.0,3.0,0.0,4.0,...,0.0,0.0,0.0,2,2.0,9.0,3.0,4,1,3


In [22]:
azdias.shape

(891221, 334)

>We went from 360 columns/features in azdias to 334 columns/features.

(Checking whether there are attributes described in attributes.csv, but not found in azdias):

In [23]:
list(set(features_in_attributes_info) - set(azdias))

[]

> Indeed, there are features described in the xlsx file (and manually created attributes_info) but that are not found in azdias, so we need to be careful when working with features/columns.

### 2. Convert Missing ('X' or 'XX') and Unknown Values of the AZDIAS features/columns to NaN values

Let's create a Pandas Series holding the values referring to 'unknown' for each feature/column of the azdias dataset.
We need, as part of the data cleaning and preprocessing step, to convert these values to NaN values.

In [24]:
unknown_series = pd.Series(attributes_info['Unknown_Values'].values, index=attributes_info['Attribute'])

In [25]:
unknown_series

Attribute
AGER_TYP                           [-1,0]
ALTERSKATEGORIE_FEIN             [-1,0.9]
ALTERSKATEGORIE_GROB             [-1,0,9]
ALTER_HH                              [0]
ANREDE_KZ                          [-1,0]
ANZ_HAUSHALTE_AKTIV                    []
ANZ_HH_TITEL                           []
ANZ_KINDER                             []
ANZ_PERSONEN                           []
ANZ_STATISTISCHE_HAUSHALTE             []
ANZ_TITEL                              []
ARBEIT                                [9]
BALLRAUM                             [-1]
CAMEO_DEUG_2015                ['-1','X']
CAMEO_DEU_2015                     ['XX']
CAMEO_INTL_2015               ['-1','XX']
CJT_GESAMTTYP                         [0]
D19_BANKEN_ANZ_12                     [0]
D19_BANKEN_ANZ_24                     [0]
D19_BANKEN_DATUM                     [10]
D19_BANKEN_DIREKT                     [0]
D19_BANKEN_GROSS                      [0]
D19_BANKEN_LOKAL                      [0]
D19_BANKEN_OFFLINE_DATUM

In [31]:
for column in azdias.columns:
    if azdias[column].dtype == np.int64:
        azdias[column] = azdias[column].astype(np.float64)

In [32]:
for column in azdias.columns:
    
    isin = ast.literal_eval(unknown_series[column])
    
    azdias[column] = azdias[column].mask(azdias[column].isin(isin), other=np.nan)

In [33]:
azdias.head()

,AGER_TYP,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,...,SOHO_KZ,TITEL_KZ,UNGLEICHENN_FLAG,VERS_TYP,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2
1,NaN,NaN,21.0,11.0,0.0,0.0,2.0,12.0,0.0,3.0,...,1.0,NaN,1.0,2.0,3.0,9.0,4.0,5.0,2.0,1
2,NaN,17.0,17.0,10.0,0.0,0.0,1.0,7.0,0.0,3.0,...,0.0,NaN,0.0,1.0,3.0,9.0,2.0,5.0,2.0,3
3,2.0,13.0,13.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,...,0.0,NaN,0.0,1.0,NaN,9.0,7.0,3.0,2.0,4
4,NaN,20.0,14.0,3.0,0.0,0.0,4.0,3.0,0.0,4.0,...,0.0,NaN,0.0,2.0,2.0,9.0,3.0,4.0,1.0,3


In [38]:
azdias.shape

(891221, 334)

In [49]:
# saving modified azdias for next time I reopen the notebook (you'll also need to rerun imports cell)
azdias.to_pickle('azdias_intermediary.pkl')

### 3. a) Identify & Drop the Missing (NaN) Values COLUMN-WISE

In [4]:
# loading saved azdias
azdias = pd.read_pickle('azdias_intermediary.pkl')

In [5]:
azdias.shape

(891221, 334)

Below are displayed the percentage of NaN values **column-wise**. We will then choose a threshold beyond which we should drop columns. This threshold must be high enough that any column with higher percentage of NaN values can be considered unuseful to our subsequent analyses.

In [11]:
# Find the percentage of missing values NaN in each column of AZDIAS, rounded to 2 decimal places
missing_perct_column = azdias.isnull()

In [9]:
missing_perct_column

,AGER_TYP,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,...,SOHO_KZ,TITEL_KZ,UNGLEICHENN_FLAG,VERS_TYP,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
6,True,True,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
7,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
8,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
9,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


> The values above are given in percentage. For example, ALTER_HH - 8.25 means that 8.25% of the values in the column 'ALTER_HH' are missing values.

In [ ]:
plt.hist(missing_perct_column, bins=20)

We decide that we only want columns (in our AZDIAS dataset) that have strictly less than 20% of percentage of missing values (column-wise percentage). Below, we will narrow down our azdias dataset to satisfy this:

In [ ]:
columns_to_drop = missing_perct_column[missing_perct_column > 0.2].index

columns_to_drop

In [ ]:
azdias.drop(labels=columns_to_drop, axis=1, inplace=True)

In [ ]:
azdias.columns

In [ ]:
# checking AZDIAS dataset shape
azdias.shape

>We went from 334 columns/features in azdias (from 2.) to 261 columns/features after 3.a)

### 3. b) Identify & Drop the Missing (NaN) Values ROW-WISE

Below are displayed the percentage of NaN values **row-wise**. We will then choose a threshold beyond which we should drop rows. This threshold must be high enough that any row with higher percentage of NaN values can be considered unuseful to our subsequent analyses.

In [41]:
# Find the percentage of missing values NaN in each row of AZDIAS, rounded to 2 decimal places
missing_perct_row = azdias.isnull().sum(axis=1)

In [ ]:
missing_perct_row

In [ ]:
plt.hist(missing_perct_row / azdias.shape[1], bins=80)

We decide that we only want rows (in our AZDIAS dataset) that have strictly less than 20% of percentage of missing values (row-wise percentage). Below, we will narrow down our azdias dataset to satisfy this:

In [ ]:
azdias = azdias[missing_perct_row / azdias.shape[1] < 0.2]

In [ ]:
azdias.columns

In [ ]:
azdias.shape

>We went from 261 columns/features in azdias (from 3.a) to 261 columns/features after 3.b)

> **no dropped rows here with our threshold set at 0.2.**

### 4. Re-Encoding Categorical & Mixed Features

Some features in the azdias dataset are categorical, some other mixed. To prepare these for feature extraction, we will need to convert these into numerical values.

**CATEGORICAL FEATURES EXPLORATION**

In [ ]:
# categorical features (attention: these features are from attributes_info, some might not be in azdias)
# recall: we dropped features in azdias that were not described in attributes_info, but there might also be
# features described in attributes_info that are not found in azdias.


categorical_features_attributes = attributes_info['Attribute'].loc[attributes_info['Type'] == 'categorical']

categorical_features_attributes # 'attributes' to indicate they're from attributes_info

In [ ]:
cat_feat_azdias = [] # categorical features (in attributes_info) that are also in azdias

for cat_feat in categorical_features_attributes:
    
    if cat_feat in azdias.columns:
        
        print(cat_feat + ', feature from attributes_info, is also a feature/column of azdias.')
        
        cat_feat_azdias.append(cat_feat)


print('There are ' + str(len(categorical_features_attributes)) + ' categorical features in attributes_info, and ' 
      + str(len(cat_feat_azdias)) + ' categorical features in azdias.')

In [ ]:
# categorical features in azdias: we will encode these categorical features
cat_feat_azdias

**MIXED FEATURES EXPLORATION**

In [ ]:
# mixed features (attention: these features are from attributes_info, some might not be in azdias)
# recall: we dropped features in azdias that were not described in attributes_info, but there might also be
# features described in attributes_info that are not found in azdias.


mixed_features_attributes = attributes_info['Attribute'].loc[attributes_info['Type'] == 'mixed']

mixed_features_attributes # 'attributes' to indicate they're from attributes_info

In [ ]:
mixed_feat_azdias = [] # mixed features (in attributes_info) that are also in azdias

for mixed_feat in mixed_features_attributes:
    
    if mixed_feat in azdias.columns:
        
        print(mixed_feat + ', feature from attributes_info, is also a feature/column of azdias.')
        
        mixed_feat_azdias.append(mixed_feat)


print('There are ' + str(len(mixed_features_attributes)) + ' mixed features in attributes_info, and ' 
      + str(len(mixed_feat_azdias)) + ' mixed features in azdias.')

To avoid confusion with the case of categorical features, we will work with mixed_feat_azdias, even though the number of mixed features in attributes_info and the number of mixed features in azdias are the same (equal to 4).

In [ ]:
# mixed features in azdias: we will encode these mixed features
mixed_feat_azdias

**1. Categorical Features to Numerical Features**

-> For binary categorical variables that take numeric values, no change is needed.

-> For binary categorical variables that take on non-numeric values, we need to re-encode the values as numbers (numerical type).

-> For multi-level categorical variables (>3 values), we need to one-hot encode the values.

In [ ]:
binary_num_attributes = []
binary_non_num_attributes = []
multi_level_attributes = []

for attribute in cat_feat_azdias:
    dtype = azdias[attribute].dtype
    count = len(azdias[attribute].value_counts())

    # if multi-level categorical feature
    if count > 2:
        multi_level_attributes.append(attribute)
    else:
        if dtype == 'object':
            binary_non_num_attributes.append(attribute)
        else:
            binary_num_attributes.append(attribute)

In [ ]:
binary_num_attributes

In [ ]:
binary_non_num_attributes

In [ ]:
multi_level_attributes

Encoding binary string categorical attribute/feature: OST_WEST_KZ

In [ ]:
# Re-encode OST_WEST_KZ binary non-numerical feature as dummy variable
# look at DIAS Attributes - Values 2017 (1) file to see what the original values are for OST_EST_KZ 
# it is: 'O' for East(GDR) and 'W' for West (FRG)

azdias['OST_WEST_KZ'] = azdias['OST_WEST_KZ'].map({'W': 1, 'O': 2})

Encoding multi-level categorical attributes/features: `multi_level_attributes`

In [ ]:
list_columns_to_add = []

for column in multi_level_attributes:
    
    list_columns_to_add.append(pd.get_dummies(azdias[column], prefix=column))
    
# drop the original multi-level categorical features 
azdias.drop(multi_level_attributes, axis=1, inplace=True)

list_columns_to_add.append(azdias) 

# add the re-encoded multi-level categorical features to azdias dataframe
azdias = pd.concat(list_columns_to_add, axis=1)

In [ ]:
azdias.head()

In [ ]:
azdias.shape

**2. Mixed Features to Numerical Features**

In [ ]:
mixed_feat_azdias

a) Re-encoding `PRAEGENDE_JUGENDJAHRE` feature

In DIAS Attributes - Values 2017 file, we see that this column/feature can take a range of value between -1 and 15 (with 0 and -1 being unknown values, thus converted to NaN). The feature PRAEGENDE_JUGENDJAHRE is indicating the 'dominating movement in the person's youth (avantgarde or mainstream)'.

The xlsx file memtioned above shows us a dichotomy, with some values belonging to 'Avantgarde' and others belonging to 'Mainstream'.

We can re-encode PRAEGENDE_JUGENDJAHRE, which will then take on either 0 or 1 as a value for each row of the dataset, with 0 refering to 'Mainstream' and 1 to ' Avantgarde'.

In [ ]:
# pj = short for PRAEGENDE_JUGENDJAHRE

def encode_pj(value): 
    
    re_encoded_value = 0
    
    # re-encoding for Mainstream movement: 0
    if value in [1, 3, 5, 8, 10, 12, 14]:
        re_encoded_value = 0
    
    # re-encoding for Avantgarde movement: 1
    elif value in [2, 4, 6, 7, 9, 11, 13, 15]:
        re_encoded_value = 1
        
    return re_encoded_value

In [ ]:
azdias['PRAEGENDE_JUGENDJAHRE'] = azdias['PRAEGENDE_JUGENDJAHRE'].apply(lambda x: encode_pj(x))

b) Dropping `LP_LEBENSPHASE_FEIN` and `LP_LEBENSPHASE_GROB` features

The DIAS Attributes - Values 2017 file (file describing features, the range of values they can take on) shows us that these 2 features/columns take on numerous values, with `LP_LEBENSPHASE_FEIN` that can take any integer value between 1 and 40 (a lot). 

As these 2 features are quite complex (relative to the rest of features), and given the number of other features at hand, we make the decision to simply drop them.

In [ ]:
# check out the 2 mentioned features
azdias['LP_LEBENSPHASE_FEIN']

In [ ]:
azdias['LP_LEBENSPHASE_GROB']

In [ ]:
# dropping the features, as decided above
azdias.drop(['LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB'], axis=1, inplace=True)

c) Re-encoding `CAMEO_INTL_2015` feature

Let's review `azdias` again:

In [ ]:
azdias.head()

In [ ]:
azdias.shape()

All-in-one Preprocessing Cleaning Function

In [ ]:
def cleaning_function(df, attributes_info, not_described_features, columns_to_drop):
    """
    Cleans and Preprocess a DataFrame df:
    - Drop features that are not described (described features summarised in attributes_info)
    - Convert missing and unknown values to NaN
    - Drop columns (so, features) with high percentage of NaN values (relative to nan_threshold)
    - Drop rows with high percentage of NaN values (relative to nan_threshold)
    - Re-encode categorical features and mixed features

    INPUT:
    - df (DataFrame): the DataFrame to be cleaned and preprocessed
    - attributes_info (DataFrame): the DataFrame containing the features information/description (Attribute, Type, Unknown_Values)
    - not_described_features (list): list of features to be dropped because they are not described (not in attributes_info)
    - columns_to_drop (list): list of columns/features to be dropped because of a high percentage of NaN values (computed in prior steps)
    
    OUTPUT:
    - df (DataFrame): cleaned and preprocessed DataFrame
    """
    
    # drop features that are not described 
    df.drop(labels=not_described_features, axis=1, inplace=True)
    
    for column in df.columns:
        if df[column].dtype == np.int64:
            df[column] = azdias[column].astype(np.float64)
    
    # convert missing and unknown values to NaN values
    for column in df.columns:
        isin = ast.literal_eval(unknown_series[column])
        df[column] = df[column].mask(df[column].isin(isin), other=np.nan)
        
    # drop columns with percentage of NaN values higher than threshold (nan_threshold)   
    df.drop(labels=columns_to_drop, axis=1, inplace=True)
    
    # drop rows with percentage of NaN values higher than threshold (nan_threshold)   
    missing_perct_row = azdias.isnull().sum(axis=1)
    df = df[missing_perct_row / df.shape[1] < nan_threshold]
    
    # categorical features of df
    cat_feat_df = [] # categorical features (in attributes_info) that are also in df
    for cat_feat in categorical_features_attributes:
        if cat_feat in df.columns:
            cat_feat_df.append(cat_feat)
    
    # mixed features of df
    mixed_feat_df = [] # mixed features (in attributes_info) that are also in df
    for mixed_feat in mixed_features_attributes:
        if mixed_feat in df.columns:
            mixed_feat_df.append(mixed_feat)

            
    ### df categorical features re-encoding ###
    binary_num_attributes = []
    binary_non_num_attributes = []
    multi_level_attributes = []

    for attribute in cat_feat_df:
        dtype = df[attribute].dtype
        count = len(df[attribute].value_counts())

        # if multi-level categorical feature
        if count > 2:
            multi_level_attributes.append(attribute)
        else:
            if dtype == 'object':
                binary_non_num_attributes.append(attribute)
            else:
                binary_num_attributes.append(attribute)
    
    # re-encode binary string (non-numerical) feature into numerical values         
    df['OST_WEST_KZ'] = df['OST_WEST_KZ'].map({'W': 1, 'O': 2})
    
    # re-encode multi-level categorical features
    list_columns_to_add = []

    for column in multi_level_attributes:
    
        list_columns_to_add.append(pd.get_dummies(df[column], prefix=column))
    
    df.drop(multi_level_attributes, axis=1, inplace=True)

    list_columns_to_add.append(df) 

    df = pd.concat(list_columns_to_add, axis=1)
    
    
    ### df mixed features re-encoding ###
    
    ??????????????????????????????????????????
    
    
    
    
    return df

### Preprocessing the CUSTOMERS Dataset with the cleaning function defined above

First, let's review the uncleaned original `customers`:

In [ ]:
customers.head()

In [ ]:
customers.shape

In [ ]:
len(customers.columns)

Cleaning and Preprocessing `customers`:

In [ ]:
# with features_in_attributes_info = attributes_info['Attribute'].tolist()

not_described_features = list(set(customers) - set(features_in_attributes_info))

In [ ]:
customers = cleaning_function(customers, not_described_features)

Now, let's look at the cleaned `customers`:

In [ ]:
customers.head()

In [ ]:
customers.shape

In [ ]:
customers.describe()

### Saving the cleaned azdias and customers datasets for next use

In [2]:
# save cleaned/preprocessed azdias dataset for next use 

azdias.to_pickle('azdias_clean.pkl')

NameError: name 'pickle' is not defined

In [ ]:
# save cleaned/preprocessed customers dataset for next use

customers.to_pickle('customers_clean.pkl')

## Impute Remaining NaN Values

Reloading cleaned and preprocessed `azdias` and `customers`

In [ ]:
azdias_clean = pd.read_pickle('azdias_clean.pkl')

In [ ]:
customers_clean = pd.read_pickle('customers_clean.pkl')

Review `azdias` and `customers`

In [ ]:
azdias_clean.head()

In [ ]:
azdias_clean.shape

In [ ]:
customers_clean.head()

In [ ]:
customers_clean.shape

Imputing the NaN Values with the mean method

In [ ]:
imputer = Imputer(strategy='median')

In [ ]:
azdias_imputed = pd.DataFrame(imputer.fit_transform(azdias_clean))

In [ ]:
customers_imputed = pd.DataFrame(imputer.fit_transform(customers_clean))

## Feature Scaling

We will now scale the remaining features: this step is critical prior to applying PCA, as principal component vectors would otherwise be influenced by the differences in scale of our data features.

In [ ]:
scaler = StandardScaler()

In [ ]:
azdias_scaled = scaler.fit_transform(azdias_imputed)

In [ ]:
customers_scaled = scaler.fit_transform(customers_imputed)

In [ ]:
azdias_scaled.shape

In [ ]:
customers_scaled.shape

In [ ]:
# For next use in Part 1 and 2

azdias_scaled.to_pickle('azdias_final.pkl')

customers_scaled.to_pickle('customers_final.pkl')

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

### First, load cleaned datasets from Part 0 

In [ ]:
# final cleaned/preprocessed azdias object

azdias_final = pd.read_pickle('data/azdias_final.pkl')

In [ ]:
# final cleaned/preprocessed customers object

customers_final = pd.read_pickle('data/customers_final.pkl')

### A. Dimensionality Reduction - PCA

In the first step of Part 1 (unsupervised learning), we will employ Principal Component Analysis, with the goal to reduce the number of features within a dataset while retaining the “principal components”, defined as weighted, linear combinations of existing features. 

These principal components are meant to be linearly independent (solving the problem of highly correlated features) and account for the largest possible variability in the data. We will soon decide on the amount of variability we think is sufficient to then reliably employ for our (next) clustering task with k-Means clustering.

### B. Clustering - k-Means Clustering

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [3]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
mailout_test

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,1754,2,1.0,7.0,NaN,NaN,NaN,NaN,6.0,2.0,...,4.0,5.0,6.0,3.0,6.0,9.0,3.0,3,1,4
1,1770,-1,1.0,0.0,NaN,NaN,NaN,NaN,0.0,20.0,...,1.0,5.0,2.0,1.0,6.0,9.0,5.0,3,1,4
2,1465,2,9.0,16.0,NaN,NaN,NaN,NaN,11.0,2.0,...,3.0,9.0,6.0,3.0,2.0,9.0,4.0,3,2,4
3,1470,-1,7.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,...,2.0,6.0,6.0,3.0,NaN,9.0,2.0,3,2,4
4,1478,1,1.0,21.0,NaN,NaN,NaN,NaN,13.0,1.0,...,1.0,2.0,4.0,3.0,3.0,9.0,7.0,4,2,4
5,1782,2,1.0,7.0,NaN,NaN,NaN,NaN,7.0,66.0,...,4.0,9.0,10.0,7.0,6.0,9.0,3.0,6,1,4
6,1485,2,1.0,10.0,NaN,NaN,NaN,NaN,9.0,1.0,...,2.0,1.0,2.0,1.0,6.0,9.0,3.0,3,2,4
7,1519,-1,1.0,20.0,NaN,NaN,NaN,NaN,15.0,1.0,...,2.0,4.0,5.0,4.0,2.0,9.0,3.0,4,2,3
8,1835,1,1.0,19.0,NaN,NaN,NaN,NaN,13.0,2.0,...,1.0,1.0,1.0,1.0,4.0,9.0,7.0,1,2,3
9,1522,1,1.0,0.0,NaN,NaN,NaN,NaN,9.0,1.0,...,2.0,5.0,8.0,9.0,3.0,4.0,2.0,4,1,3
